In [39]:
import numpy as np
import pandas as pd
import sklearn.preprocessing 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
import nltk
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
sns.set()

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TUKL summer internship/week 2/Supreme Court Data 5.csv')
df.head()

,Case Files,Label,Category
0,2017 P T D 1481\n[Supreme Court of Pakistan]\n...,Non-Bailable,Against Federation
1,2015 S C M R 365\n[Supreme Court of Pakistan]\...,Bailable,illegal appointments
2,2016 S C M R 1420\n[Supreme Court of Pakistan]...,Bailable,Theft/Fraud
3,2016 S C M R 662\n[Supreme Court of Pakistan]\...,Bailable,Theft/Fraud
4,P L D 2012 Supreme Court 649\nPresent: Mian Sh...,Bailable,illegal appointments


In [42]:
df1 = df[['Case Files','Label']]
df1.isnull().sum()

Case Files    0
Label         0
dtype: int64

In [43]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [45]:
def cleaning(df, stop_words):

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join(x.lower() for x in x.split())) 

                               

    # Replacing the digits/numbers

    df['Case Files'] = df['Case Files'].str.replace('d', '')

            

    # Removing stop words

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))

    #lower casing
    df['Case Files'] = df['Case Files'].str.lower()

    #punctuation removal         
    df["Case Files"] = df["Case Files"].apply(lambda text: remove_punctuation(text))
    # Lemmatization

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))

            

    return df

stop_words = stopwords.words('english')

df1 = cleaning(df1, stop_words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [46]:
df2 = df1.rename(columns={'Case Files': 'Case_Files'})
df2.head()

,Case_Files,Label
0,2017 p 1481 supreme court pakistan present mia...,Non-Bailable
1,2015 c r 365 supreme court pakistan present na...,Bailable
2,2016 c r 1420 supreme court pakistan present a...,Bailable
3,2016 c r 662 supreme court pakistan present an...,Bailable
4,p l 2012 supreme court 649 present mian shakir...,Bailable


In [47]:
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df2['Case_Files'].values)
X = tokenizer.texts_to_sequences(df2['Case_Files'].values)
X = pad_sequences(X)

In [48]:
df2['encoded']=df2['Label'].apply(lambda x: 1 if x=='Bailable' else 0)
df2.head()

,Case_Files,Label,encoded
0,2017 p 1481 supreme court pakistan present mia...,Non-Bailable,0
1,2015 c r 365 supreme court pakistan present na...,Bailable,1
2,2016 c r 1420 supreme court pakistan present a...,Bailable,1
3,2016 c r 662 supreme court pakistan present an...,Bailable,1
4,p l 2012 supreme court 649 present mian shakir...,Bailable,1


In [49]:
df2['encoded']

0     0
1     1
2     1
3     1
4     1
     ..
61    1
62    0
63    0
64    1
65    1
Name: encoded, Length: 66, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split#stratify ensures balance between spam and no spam in test and train
xtrain, xtest, ytrain, ytest =train_test_split(X, df2['encoded'], 
                                random_state=2018, 
                                test_size=0.3, 
                                stratify=df2['encoded'])

In [51]:
from sklearn.ensemble import AdaBoostClassifier
MLmodel1=AdaBoostClassifier(random_state=100)
from sklearn.tree import DecisionTreeClassifier
MLmodel2 = DecisionTreeClassifier(random_state=100,criterion='gini',min_samples_split=2,splitter='random',max_depth=10)
from sklearn.ensemble import RandomForestClassifier
MLmodel3  = RandomForestClassifier(random_state=100,n_estimators=1,min_samples_split=10)
from sklearn.ensemble import AdaBoostClassifier
MLmodel4=AdaBoostClassifier(random_state=100)

In [53]:
MLmodel1.fit(xtrain,ytrain)
MLmodel2.fit(xtrain,ytrain)
MLmodel3.fit(xtrain,ytrain)
MLmodel4.fit(xtrain,ytrain)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=100)

In [54]:
predict1=MLmodel1.predict(xtest)
predict2=MLmodel2.predict(xtest)
predict3=MLmodel3.predict(xtest)
predict4=MLmodel4.predict(xtest)

In [55]:
from sklearn.metrics import accuracy_score
accuracy1=accuracy_score(ytest,predict1)
accuracy2=accuracy_score(ytest,predict2)
accuracy3=accuracy_score(ytest,predict3)
accuracy4=accuracy_score(ytest,predict4)
print(accuracy1)
print(accuracy2)
print(accuracy3)
print(accuracy4)

0.55
0.5
0.5
0.55
